[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/costaalv/descobrindo_caos/blob/main/notebooks/11_anatomia_autovetores.ipynb)

# 11 · A Anatomia do Caos - Análise dos Autovetores

**Registro observacional associado ao livro**  
*Descobrindo o Caos nos Números — Como a ordem emerge quando mudamos a forma de observar*  
© Alvaro Costa, 2025  

Este notebook faz parte de uma sequência canônica de registros computacionais.  
Ele não introduz hipóteses, conjecturas ou modelos interpretativos novos.

Seu objetivo é exclusivamente **registrar** o comportamento de estruturas aritméticas sob um regime  
de observação explícito, determinístico e reproduzível.

A leitura conceitual completa encontra-se no livro. Este notebook documenta apenas o experimento  
correspondente.

**Licença:** Creative Commons BY–NC–ND 4.0  
É permitida a leitura, execução e citação.  
Não é permitida a modificação, redistribuição adaptada ou uso comercial independente.

---

## 1. Para além dos autovalores

Nos capítulos anteriores, a análise concentrou-se nos **autovalores** $ \lambda_i $ do operador $ M $.  
Eles permitiram caracterizar **onde** as excitações espectrais se localizam e, por meio das estatísticas de espaçamento, distinguir entre  
regimes descorrelacionados (Poisson) e correlacionados (GOE).

Neste notebook, o foco desloca-se para os **autovetores** $ v_i $.

Se os autovalores descrevem a estrutura espectral global, os autovetores revelam **como essa estrutura se manifesta internamente**.  
Eles codificam os padrões de “vibração” do operador e permitem verificar se o regime observado é apenas estatisticamente compatível ou  
estruturalmente **ergódico**.

---

## 2. Ferramenta I — Participation Ratio (PR)

O *Participation Ratio* (PR) quantifica o grau de **deslocalização** de um autovetor.

Para um autovetor normalizado $ v = (v_1,\dots,v_N) $, define-se:

$
\mathrm{PR} = \frac{1}{\sum_{i=1}^{N} |v_i|^4}.
$

Interpretação:

- **Autovetor localizado**: concentrado em poucas componentes → PR pequeno (ordem da unidade).
- **Autovetor deslocalizado**: espalhado por todo o espaço → PR proporcional a $ N $.

Na Teoria de Matrizes Aleatórias, autovetores do **GOE** são ergódicos.  
Nesse caso, a razão normalizada

$
\frac{\mathrm{PR}}{N}
$

concentra-se em torno do valor universal:

$
\frac{1}{3}.
$

Esse valor é uma **assinatura estrutural** de caos quântico, independente da forma detalhada do operador.

---

## 3. Ferramenta II — Estatística dos componentes

Uma segunda previsão independente da Teoria de Matrizes Aleatórias diz respeito aos **componentes individuais** dos autovetores.

Para matrizes da classe GOE, os componentes de um autovetor típico do *bulk* comportam-se como variáveis aleatórias extraídas de  
uma **distribuição Gaussiana real**, com média zero.

Neste notebook, essa previsão é testada da seguinte forma:

1. Seleciona-se um autovetor do centro do espectro.
2. Constrói-se o histograma de seus componentes.
3. Compara-se o histograma com uma curva Gaussiana teórica.
4. Aplica-se o **teste de Kolmogorov–Smirnov (KS)** para quantificar a compatibilidade.

Um *p-valor* elevado indica que não há evidência estatística para rejeitar a hipótese Gaussiana.

---

## 4. O laboratório dos autovetores

As células de código a seguir implementam essas duas análises de forma comparativa.

São gerados quatro gráficos:

- **Escala linear**:
  - histograma dos componentes de um autovetor;
  - distribuição dos valores de $ \mathrm{PR}/N $.

- **Escala logarítmica**:
  - histograma dos componentes de um autovetor;
  - distribuição dos valores de $ \mathrm{PR}/N $.

O objetivo é verificar se a distinção observada nos autovalores (Poisson versus GOE) também se manifesta na **geometria interna**  
dos autovetores.

A interpretação conceitual desses resultados é apresentada no texto do Capítulo 11.


In [1]:
# Requisitos: pandas, matplotlib, numpy, ipywidgets, scipy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time
from scipy.stats import kstest, norm

# --- Funções de Geração de Dados e Matriz (do capítulo anterior) ---
def generate_pi_data(n: int) -> np.ndarray:
    """Gera um array com todos os primos até n usando um crivo otimizado."""
    if n < 2: return np.array([], dtype=np.int64)
    size = (n - 1) // 2; sieve = np.ones(size, dtype=bool)
    limit = int(np.sqrt(n)) // 2
    for i in range(limit):
        if sieve[i]:
            p = 2 * i + 3; start = (p*p - 3) // 2
            sieve[start::p] = False
    indices = np.where(sieve)[0]; odd_primes = 2 * indices + 3
    return np.concatenate((np.array([2], dtype=np.int64), odd_primes))

def get_delta_pi_for_points(x_points, primes):
    """Calcula Δπ(x) para um array de pontos x usando uma lista de primos pré-calculada."""
    x_int = np.floor(x_points).astype(int)
    pi_x = np.searchsorted(primes, x_int, side='right')
    pi_x_div_2 = np.searchsorted(primes, x_int // 2, side='right')
    return pi_x - 2 * pi_x_div_2
    
def generate_cos_matrix(fx_values, x_values):
    fx = fx_values.astype(np.float64); x = x_values.astype(np.float64)
    x[x <= 0] = 1e-12; logx = np.log(x)
    C = np.cos(np.outer(fx, logx)); M = C + C.T
    std_dev = M.std()
    if std_dev > 0: M -= M.mean(); M /= std_dev
    return 0.5 * (M + M.T)

# FUNÇÃO PARA ANÁLISE DE AUTOVETORES
def calculate_participation_ratio(eigenvectors):
    """Calcula o Participation Ratio para uma matriz de autovetores."""
    # Autovetores de eigh são normalizados (soma dos quadrados é 1)
    # A fórmula é PR = 1 / sum(|v_i|^4)
    return 1 / np.sum(eigenvectors**4, axis=0)

# --- A Função Interativa Principal ---
def eigenvector_analysis_lab(N=2048, log_X0=7):
    
    X0 = int(10**log_X0)
    span = 2.4 # Span fixo para a escala logarítmica
    
    # --- Preparação dos Dados ---
    max_x_log = int(np.ceil(X0 * np.exp(span/2)))
    max_x_linear = X0 + N
    max_x_needed = max(max_x_log, max_x_linear)
    pi_x_full = generate_pi_data(max_x_needed)

    # --- Análise da Escala Linear ---
    print("\n--- Processando Escala Linear ---")
    x_linear = np.arange(X0, X0 + N)
    fx_linear = get_delta_pi_for_points(x_linear, pi_x_full)
    # Jitter para garantir que o método funcione
    jitter = 1e-9 * (fx_linear.std() if fx_linear.std() > 0 else 1)
    fx_linear_jittered = fx_linear.astype(np.float64) + np.random.uniform(-jitter, jitter, size=fx_linear.shape)
    M_linear = generate_cos_matrix(fx_linear_jittered, x_linear)
    lam_linear, v_linear = np.linalg.eigh(M_linear)
    
    # Análises do autovetor
    vec_linear = v_linear[:, N // 2] # Pega um autovetor do meio
    pr_linear = calculate_participation_ratio(v_linear)
    ks_linear = kstest(vec_linear * np.sqrt(N), 'norm') # Componentes devem ser normalizados por sqrt(N)

    # --- Análise da Escala Logarítmica ---
    print("\n--- Processando Escala Logarítmica ---")
    x_log = np.exp(np.linspace(np.log(X0) - span/2, np.log(X0) + span/2, N))
    fx_log = get_delta_pi_for_points(x_log, pi_x_full)
    M_log = generate_cos_matrix(fx_log, x_log)
    lam_log, v_log = np.linalg.eigh(M_log)

    # Análises do autovetor
    vec_log = v_log[:, N // 2] # Pega um autovetor do meio
    pr_log = calculate_participation_ratio(v_log)
    ks_log = kstest(vec_log * np.sqrt(N), 'norm')

    # --- Plots Comparativos (Layout 2x2) ---
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f"Análise de Autovetores para N={N}, X₀≈{X0:g}", fontsize=18, weight='bold')

    # --- Linha de Cima: Escala Linear (Poisson) ---
    # a) Componentes do Autovetor
    ax = axes[0, 0]
    ax.hist(vec_linear * np.sqrt(N), bins=50, density=True, alpha=0.7, label='Componentes')
    ax.set_title(f'a) Componentes de $v_i$ (Linear)', fontsize=14)
    ax.set_xlabel(r'Componentes $v_i \sqrt{N}$')
    ax.set_ylabel('Densidade')
    ax.text(0.05, 0.95, f'KS p-valor: {ks_linear.pvalue:.2f}', transform=ax.transAxes, ha='left', va='top', color='red')

    # b) Participation Ratio
    ax = axes[0, 1]
    ax.hist(pr_linear / N, bins=50, density=True, alpha=0.7, label='PR/N')
    mean_pr_linear = np.mean(pr_linear / N)
    ax.axvline(mean_pr_linear, color='red', ls='--', label=f'Média = {mean_pr_linear:.3f}')
    ax.set_title(f'b) Participation Ratio (Linear)', fontsize=14)
    ax.set_xlabel('PR / N')
    ax.legend()
    
    # --- Linha de Baixo: Escala Logarítmica (GOE) ---
    # c) Componentes do Autovetor
    ax = axes[1, 0]
    x_grid = np.linspace(-4, 4, 200)
    ax.hist(vec_log * np.sqrt(N), bins=50, density=True, alpha=0.7, label='Componentes')
    ax.plot(x_grid, norm.pdf(x_grid), 'r--', lw=2, label='Teoria Gaussiana')
    ax.set_title(f'c) Componentes de $v_i$ (Logarítmico)', fontsize=14)
    ax.set_xlabel(r'Componentes $v_i \sqrt{N}$')
    ax.set_ylabel('Densidade')
    ax.text(0.05, 0.95, f'KS p-valor: {ks_log.pvalue:.2f}', transform=ax.transAxes, ha='left', va='top', color='green')
    ax.legend()

    # d) Participation Ratio
    ax = axes[1, 1]
    ax.hist(pr_log / N, bins=50, density=True, alpha=0.7, label='PR/N')
    mean_pr_log = np.mean(pr_log / N)
    ax.axvline(mean_pr_log, color='red', ls='--', label=f'Média = {mean_pr_log:.3f}')
    ax.axvline(1/3, color='green', ls=':', lw=3, label='Teórico GOE ≈ 0.333')
    ax.set_title(f'd) Participation Ratio (Logarítmico)', fontsize=14)
    ax.set_xlabel('PR / N')
    ax.legend()

    fig.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# --- Widget Interativo ---
interact(eigenvector_analysis_lab, 
         N=widgets.Dropdown(options=[512, 1024, 2048], value=1024, description='N:'),
         log_X0=widgets.IntSlider(min=5, max=8, step=1, value=7, description='X₀=10^', continuous_update=False)
        );

interactive(children=(Dropdown(description='N:', index=1, options=(512, 1024, 2048), value=1024), IntSlider(va…

---

## 5. Análise dos Resultados: O Sinal Genuíno e o Artefato

Os resultados obtidos no laboratório de autovetores são, à primeira vista, paradoxais. No entanto, esse paradoxo não representa uma inconsistência  
do método — ele revela, de forma particularmente clara, a importância decisiva da **lente de observação** utilizada ao longo de todo o trabalho.

### Observação para a Escala Logarítmica

*(O sinal genuíno do caos)*

* **Componentes Gaussianos (Gráfico c)**
  O histograma dos componentes do autovetor extraído do *bulk* do espectro ajusta-se de forma consistente à curva Gaussiana teórica.  
  O teste de Kolmogorov–Smirnov fornece um p-valor típico da ordem de `0.2`, significativamente acima do limiar usual de rejeição (`0.05`).  
  Isso indica que **não há evidência estatística para rejeitar a hipótese Gaussiana**, em concordância com as previsões da Teoria de Matrizes  
Aleatórias para sistemas do tipo GOE.

* **Participation Ratio próximo de 1/3 (Gráfico d)**
  A distribuição dos valores normalizados de PR/N encontra-se fortemente concentrada em torno do valor teórico `1/3`, com médias empíricas próximas  
(por exemplo, `≈ 0.32`).
  Isso indica que os autovetores são **deslocalizados e ergódicos**, ocupando de forma homogênea todo o espaço vetorial — uma assinatura robusta de
caos quântico.

Esses dois resultados independentes confirmam que, na escala logarítmica, o operador ( M ) exibe não apenas estatísticas espectrais compatíveis com  
o GOE, mas também a **estrutura interna dos autovetores** esperada para sistemas caóticos genuínos.

---

### Observação para a Escala Linear

*(O “fantasma na máquina”)*

* **Componentes “perfeitamente” Gaussianos (Gráfico a)**
  De forma aparentemente contraintuitiva, o teste KS aplicado aos componentes dos autovetores na escala linear frequentemente produz p-valores ainda
mais elevados (por exemplo, `≈ 0.7`).
  Isoladamente, isso poderia sugerir um comportamento ainda mais “caótico”.

* **Participation Ratio elevado (Gráfico b)**
  A média de PR/N também se aproxima de `1/3`, valor típico de vetores totalmente deslocalizados.

Esse conjunto de resultados cria um paradoxo apenas aparente.

---

### A explicação do paradoxo: o efeito do *jitter*

Na escala linear, a matriz $ M $ construída a partir de $ \Delta_\pi(x) $ é estruturalmente pobre e altamente degenerada.
Para tornar o problema numericamente tratável, é necessário introduzir uma perturbação aleatória mínima (*jitter*).

O efeito dessa perturbação é decisivo:

> os autovetores passam a refletir predominantemente as propriedades estatísticas do ruído introduzido, e não mais a estrutura aritmética dos primos.

Um vetor aleatório genérico possui, por construção:

* componentes aproximadamente Gaussianos;
* Participation Ratio próximo de `1/3`.

Assim, no regime linear, estamos a medir **um artefato do método de regularização**, e não um sinal físico ou aritmético genuíno.

O contraste com a escala logarítmica é crucial: nela, a matriz $ M $ já nasce rica e complexa, dispensando qualquer *jitter*.
Mesmo assim, exibe as mesmas assinaturas — o que demonstra que o comportamento GOE observado nesse caso é **intrínseco**, e não induzido artificialmente.

---

## 6. Perspectivas Futuras: Uma Primitiva para Criptografia?

As propriedades espectrais e vetoriais observadas sugerem aplicações que vão além da análise teórica, incluindo possíveis conexões com criptografia.

A criptografia moderna baseia-se em problemas matemáticos que são computacionalmente fáceis numa direção, mas extremamente difíceis de inverter — como  
a fatoração de números primos.

Os resultados deste capítulo apontam para a possibilidade de uma **primitiva criptográfica inspirada em caos quântico**:

* **Ideia central**
  Os parâmetros do sistema — o ponto inicial $ X_0 $, o tamanho $ N $ e o funcional $ \Delta_\pi(x) $ — poderiam atuar como uma **chave privada**.  
  A partir deles, seria computacionalmente simples gerar autovetores específicos da matriz $ M $.

* **Segurança potencial**
  Esses autovetores são objetos matemáticos de altíssima complexidade, com estrutura pseudo-aleatória e extrema sensibilidade aos parâmetros iniciais.  
  O problema inverso — reconstruir $ X_0 $ ou a estrutura aritmética subjacente a partir de um autovetor — parece, à primeira vista, computacionalmente
intratável.

* **Natureza “quântica”**
  O caráter “quântico” dessa proposta não reside no uso de hardware quântico, mas no fato de que as **estatísticas envolvidas coincidem com aquelas  
observadas em sistemas físicos quânticos caóticos**, conforme a lei de Montgomery–Odlyzko.

Essas possibilidades não são exploradas neste trabalho, mas indicam uma direção promissora: a instrumentalização direta da matemática do caos quântico,  
emergente da teoria dos números, em aplicações tecnológicas concretas.
